<h1> Wage Estimator (Machine Learning HW5)
    <h4> submitted by: - Ujjawal Gupta and Dhaval Pathak

In [118]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold, cross_val_score, cross_val_predict
import matplotlib.pyplot as plt
import math

<h2> Loading The Training dataset

In [4]:
cols = ['idnum', 'age', 'workerclass', 'interestincome', 'traveltimetowork', 'vehicleoccupancy', 'meansoftransport',
        'marital', 'schoolenrollment', 'educationalattain', 'sex', 'workarrivaltime', 'hoursworkperweek', 'ancestry',
        'degreefield', 'industryworkedin', 'wages']
df_train = pd.DataFrame()
df_load = pd.read_csv("census_train.csv", header=None)
c = 0
for i in cols:
    df_train[i] = df_load[c]
    c += 1

In [5]:
for i in range(len(df_train.columns)):
    df_train[df_train.columns[i]] = df_train[df_train.columns[i]].astype(str).replace({'?': 0})

In [6]:
df_train.head()

,idnum,age,workerclass,interestincome,traveltimetowork,vehicleoccupancy,meansoftransport,marital,schoolenrollment,educationalattain,sex,workarrivaltime,hoursworkperweek,ancestry,degreefield,industryworkedin,wages
0,0,58,3,0,40,2,1,1,1,22,1,84,40,32,2300,7870,110000
1,1,34,3,0,10,1,1,5,1,16,1,168,40,51,0,7860,34500
2,2,57,2,0,15,8,1,5,1,16,1,100,30,51,0,8390,4700
3,3,17,0,0,0,0,0,5,2,15,2,0,0,714,0,0,0
4,4,42,1,0,20,1,1,1,1,22,1,92,45,21,5001,6990,120000


In [7]:
# (df_train == 0).astype(int).sum(axis=0)

In [8]:
df_train[["idnum", "age", "interestincome", "wages", "hoursworkperweek", "traveltimetowork"]] = df_train[["idnum", "age", "interestincome", "wages", "hoursworkperweek", "traveltimetowork"]].apply(pd.to_numeric)
df_train.dtypes

idnum                 int64
age                   int64
workerclass          object
interestincome        int64
traveltimetowork      int64
vehicleoccupancy     object
meansoftransport     object
marital              object
schoolenrollment     object
educationalattain    object
sex                  object
workarrivaltime      object
hoursworkperweek      int64
ancestry             object
degreefield          object
industryworkedin     object
wages                 int64
dtype: object

In [9]:
df_train2 = df_train.copy()

<h2> Dropping the unnecessary features

In [10]:
df_train = df_train.drop(['vehicleoccupancy'], axis=1)
df_train = df_train.drop(['workarrivaltime'], axis=1)
df_train = df_train.drop(['meansoftransport'], axis=1)
df_train = df_train.drop(['marital'], axis=1)
df_train = df_train.drop(['schoolenrollment'], axis=1)
df_train = df_train.drop(['ancestry'], axis=1)
df_train = df_train.drop(['age'], axis=1)
df_train = df_train.drop(['sex'], axis=1)
df_train = df_train.drop(['degreefield'], axis=1)
df_train = df_train.drop(['traveltimetowork'], axis=1)
df_train = df_train.drop(['industryworkedin'], axis=1)

<h2> One hot encoding selected Object dtype features

In [11]:
df_train = pd.get_dummies(df_train)

In [12]:
df_train.shape

(1184, 24)

In [13]:
df_train.head()

,idnum,interestincome,hoursworkperweek,wages,workerclass_0,workerclass_1,workerclass_2,workerclass_3,workerclass_4,workerclass_5,...,educationalattain_15,educationalattain_16,educationalattain_17,educationalattain_18,educationalattain_19,educationalattain_20,educationalattain_21,educationalattain_22,educationalattain_23,educationalattain_24
0,0,0,40,110000,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,0,40,34500,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
2,2,0,30,4700,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,3,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,4,0,45,120000,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


<h2>Converting Feature data and Wages into an Numpy Array

In [14]:
wages = np.array(df_train['wages'])

In [15]:
df_train = df_train.drop(['wages'], axis=1)
df_train = df_train.drop(['idnum'], axis=1)

In [16]:
df_train_columns = list(df_train.columns)

In [17]:
df_train = np.array(df_train)

<h2> Spliting the Data is Training and Test

In [18]:
df_train1, df_test, df_train1_wages, df_test_wages = train_test_split(df_train, wages, test_size = 0.25, random_state = 42)

<h2>Importing Random Forest libraries

In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


<h2> Defining parameter ranges for hyperparameter tuning

In [20]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

<h2> Running the function finding the best parameters for selected feature set

In [39]:
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, 
#                                verbose=2, random_state=42, n_jobs = -1)

In [40]:
# rf_random.best_params_

In [21]:
rf = RandomForestRegressor(n_estimators = 2000,
 min_samples_split = 10,
 min_samples_leaf = 2,
 max_features = 'auto',
 max_depth = 40,
 bootstrap = True)

<h2> Applying Cross Validation

In [31]:
for k in range(2,11):
    result = cross_val_score(rf, X=df_train1, y=df_train1_wages, cv=k, n_jobs=1, scoring='neg_mean_squared_error')
    print(k, result.mean())

2 -4009898254.293438
3 -3560552958.614225
4 -3749253073.1771336
5 -3678688278.660408
6 -3629906982.4460006
7 -3485695954.4338584
8 -3635289352.2853074
9 -3509990629.229298
10 -3541033845.0760794


In [564]:
# rf_random.best_params_

{'n_estimators': 2000,
 'min_samples_split': 10,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': 40,
 'bootstrap': True}

<h2> Calculating the result on test set

In [51]:
rf.fit(df_train1,df_train1_wages)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=40,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=10,
           min_weight_fraction_leaf=0.0, n_estimators=2000, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [52]:
predictions = rf.predict(df_test)

In [53]:
errors = abs(predictions - df_test_wages)

In [54]:
print('Mean Absolute Error:', round(np.mean(errors), 2),)

Mean Absolute Error: 22120.81


In [55]:
s = 0
for i,j in zip(df_test_wages, predictions):
    s += (i-j)**2
print(math.sqrt(s/len(df_test_wages)))
print("Mean squared error: %.2f" % mean_squared_error(df_test_wages, predictions))

62172.183294317656
Mean squared error: 3865380375.58


<h2> Linear Regression
    <h4> Dropping the unnecesary features

In [56]:
df_train2 = df_train2.drop(['vehicleoccupancy'], axis=1)
df_train2 = df_train2.drop(['workarrivaltime'], axis=1)
df_train2 = df_train2.drop(['meansoftransport'], axis=1)
df_train2 = df_train2.drop(['marital'], axis=1)
df_train2 = df_train2.drop(['schoolenrollment'], axis=1)
df_train2 = df_train2.drop(['ancestry'], axis=1)
df_train2 = df_train2.drop(['degreefield'], axis=1)
df_train2 = df_train2.drop(['educationalattain'], axis=1)
df_train2 = df_train2.drop(['industryworkedin'], axis=1)
df_train2 = df_train2.drop(['workerclass'], axis=1)
df_train2 = df_train2.drop(['sex'], axis=1)
df_train2 = df_train2.drop(['traveltimetowork'], axis=1)

In [57]:
# df_train2['traveltimetowork'] = df_train2['traveltimetowork'].fillna(df_train2['traveltimetowork'].median())
# df_train2['hoursworkperweek'] = df_train2['hoursworkperweek'].fillna(df_train2['hoursworkperweek'].median())
# # df_train['wages'] = df_train['wages'].replace({'0' : str(df_train['wages'].median())})

In [58]:
# df_train2[(df_train2 != 0).all(1)]
# # for i in range(len(df_train2.columns)):
# #     df_train2['wages'] = df_train2['wages'].replace({'0': None})
# # df_train2 = df_train2.dropna()

In [59]:
df_train_scale = pd.DataFrame()
# df_train_scale['age'] = df_train2['age']
df_train_scale['interestincome'] = df_train2['interestincome']
df_train_scale['hoursworkperweek'] = df_train2['hoursworkperweek']
df_train_scale['wages'] = df_train2['wages']
# df_train_scale['sex'] = df_train['sex']
df_train_final = df_train_scale
# scaler = MinMaxScaler()
# df_train_final[['wages']] = scaler.fit_transform(df_train_final[['wages']])
# df_train_temp = pd.DataFrame()
# cat.remove('sex')
# for i in cat:
#     lb = LabelBinarizer()
#     lb_results = lb.fit_transform(df_train[i])
#     cols = []
#     for j in lb.classes_:
#         cols.append(i + j)
#     df_train_temp = pd.DataFrame(lb_results, columns=cols)
#     df_train_final = df_train_final.join(df_train_temp)
df_train_final.head()

,interestincome,hoursworkperweek,wages
0,0,40,110000
1,0,40,34500
2,0,30,4700
3,0,0,0
4,0,45,120000


<h2>Training the model and calculating MSE

In [60]:
df_wages = df_train_final['wages']
df_train_final = df_train_final.drop(['wages'], axis=1)

In [61]:
scaler = MinMaxScaler()
df_train_final = scaler.fit_transform(df_train_final)
df_train_1, df_test_1, wages_train_1, wages_test_1 = train_test_split(df_train_final, df_wages, test_size=0.2, random_state=42)

In [62]:
regr = linear_model.LinearRegression()
regr.fit(df_train_1, wages_train_1)
pred = regr.predict(df_test_1)
s = 0
for i,j in zip(wages_test_1, pred):
    s += (i-j)**2
print(math.sqrt(s/len(wages_test_1)))
print("Mean squared error: %.2f" % mean_squared_error(wages_test_1, pred))

73994.45894184745
Mean squared error: 5475179954.10


In [14]:
# import matplotlib.pyplot as plt
# df_temp = pd.DataFrame()
# df_temp['traveltimetowork'] = df_train['traveltimetowork']
# df_temp['wages'] = df_wages
# df_temp = df_temp.dropna()
# x = df_temp['traveltimetowork']
# y = df_temp['wages']
# plt.plot(x, y)
# plt.show()

In [15]:
# ['workerclass','vehicleoccupancy','meansoftransport','marital','schoolenrollment','educationalattain','sex','workarrivaltime','ancestry','degreefield','industryworkedin']:

In [16]:
# from sklearn.cross_validation import train_test_split
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# df_train = scaler.fit_transform(df_train)
# df_train, df_test, wages_train, wages_test = train_test_split(df_train, df_wages, test_size=0.2, random_state=42)

<h2>Prediction Using Random Forest Regressor

In [103]:
 cols = ['idnum', 'age', 'workerclass', 'interestincome', 'traveltimetowork', 'vehicleoccupancy', 'meansoftransport',
        'marital', 'schoolenrollment', 'educationalattain', 'sex', 'workarrivaltime', 'hoursworkperweek', 'ancestry',
        'degreefield', 'industryworkedin']
test_df = pd.DataFrame()
df_load = pd.read_csv("census_test.csv", header=None)
c = 0
for i in cols:
    test_df[i] = df_load[c]
    c += 1

In [104]:
for i in range(len(test_df.columns)):
    test_df[test_df.columns[i]] = test_df[test_df.columns[i]].astype(str).replace({'?': 0})

In [105]:
test_df[["idnum", "age", "interestincome", "hoursworkperweek", "traveltimetowork"]] = test_df[["idnum", "age", "interestincome", "hoursworkperweek", "traveltimetowork"]].apply(pd.to_numeric)
test_df.dtypes

idnum                 int64
age                   int64
workerclass          object
interestincome        int64
traveltimetowork      int64
vehicleoccupancy     object
meansoftransport     object
marital              object
schoolenrollment     object
educationalattain    object
sex                  object
workarrivaltime      object
hoursworkperweek      int64
ancestry             object
degreefield          object
industryworkedin     object
dtype: object

In [106]:
test_df = test_df.drop(['vehicleoccupancy'], axis=1)
test_df = test_df.drop(['workarrivaltime'], axis=1)
test_df = test_df.drop(['meansoftransport'], axis=1)
test_df = test_df.drop(['marital'], axis=1)
test_df = test_df.drop(['schoolenrollment'], axis=1)
test_df = test_df.drop(['ancestry'], axis=1)
test_df = test_df.drop(['age'], axis=1)
test_df = test_df.drop(['sex'], axis=1)
test_df = test_df.drop(['degreefield'], axis=1)
test_df = test_df.drop(['traveltimetowork'], axis=1)
test_df = test_df.drop(['industryworkedin'], axis=1)

In [107]:
test_df = pd.get_dummies(test_df)
test_df.shape

(500, 23)

In [108]:
idnum = test_df['idnum']

In [109]:
test_df = test_df.drop(['idnum'], axis=1)

In [110]:
test_df = np.array(test_df)

In [111]:
Final_Predictions = rf.predict(test_df)

In [112]:
Pred = pd.DataFrame(Final_Predictions, columns=['Wages'])

In [113]:
result = pd.concat([idnum, Pred], axis=1, join='inner')

In [115]:
result.rename(columns={'idnum': 'Id'}, inplace=True)

<h2> Storing the results on the output file

In [117]:
result.to_csv("test_outputs.csv", index = False, sep=',', encoding='utf-8')